<a href="https://colab.research.google.com/github/EfeTascioglu/hackTheNorth2019/blob/master/QuakeTester.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import xgboost as xgb
import math
import requests
import datetime as dt
from datetime import datetime
import sklearn as sk
from datetime import datetime

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn import preprocessing
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
!wget https://github.com/efetascioglu/hackTheNorth2019/raw/master/query1.csv

--2019-09-15 03:20:18--  https://github.com/efetascioglu/hackTheNorth2019/raw/master/query1.csv
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/efetascioglu/hackTheNorth2019/master/query1.csv [following]
--2019-09-15 03:20:18--  https://raw.githubusercontent.com/efetascioglu/hackTheNorth2019/master/query1.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17498761 (17M) [text/plain]
Saving to: ‘query1.csv’

query1.csv          100%[===================>]  16.69M  84.7MB/s    in 0.2s    

2019-09-15 03:20:18 (84.7 MB/s) - ‘query1.csv’ saved [17498761/17498761]

